# 2 register device to fiware on minikube

**confirm that you have been started `iotagent-json`**

change ${CORE_ROOT} to your path of `core`.

In [ ]:
export CORE_ROOT="${HOME}/core"

change ${PJ_ROOT} to your path of `uoa-poc2`.

In [ ]:
export PJ_ROOT="${HOME}/uoa-poc2"
cd ${PJ_ROOT};pwd

example)
```
/Users/user/uoa-poc2
```

## load environment variables

load from `core`

In [ ]:
source ${CORE_ROOT}/docs/environments/minikube/env

load from `uoa-poc2`

In [ ]:
source ${PJ_ROOT}/docs/environments/minikube/env

## setup alias

In [ ]:
alias now="python -c 'import datetime; print(datetime.datetime.now().strftime(\"%Y-%m-%dT%H:%M:%S.%f+09:00\"))'"
alias iso8601="python -c 'import datetime; print(datetime.datetime.now(tz=datetime.timezone.utc).isoformat(timespec=\"seconds\"))'"

### register users to RabbitMQ

In [ ]:
for e in $(env); do
  if [[ "${e}" =~ ^MQTT__([[:alnum:]_-]+)=([[:alnum:]_-]+)$ ]]; then
    username=${BASH_REMATCH[1]}
    password=${BASH_REMATCH[2]}
    
    kubectl exec rabbitmq-0 -- rabbitmqctl add_user ${username} ${password}
    kubectl exec rabbitmq-0 -- rabbitmqctl set_permissions -p / ${username} ".*" ".*" ".*"
  fi
done

In [ ]:
kubectl exec rabbitmq-0 -- rabbitmqctl list_users

example)
```
Listing users ...
user	tags
ros	[]
guest	[administrator]
iotagent	[]
```

## register delivery robot service

In [ ]:
TOKEN=$(cat ${CORE_ROOT}/secrets/auth-tokens.json | jq '.[0].settings.bearer_tokens[0].token' -r)
curl -H "Authorization: bearer ${TOKEN}" -H "Fiware-Service: ${FIWARE_SERVICE}" -H "Fiware-ServicePath: ${DELIVERY_ROBOT_SERVICEPATH}" -H "Content-Type: application/json" http://${HOST_IPADDR}:8080/idas/json/manage/iot/services/ -X POST -d @- <<__EOS__
{
  "services": [
    {
      "apikey": "${DELIVERY_ROBOT_TYPE}",
      "cbroker": "http://orion:1026",
      "resource": "/iot/json",
      "entity_type": "${DELIVERY_ROBOT_TYPE}"
    }
  ]
}
__EOS__

expected)
```json
{}
```

### confirm registered service

In [ ]:
TOKEN=$(cat ${CORE_ROOT}/secrets/auth-tokens.json | jq '.[0].settings.bearer_tokens[0].token' -r)
curl -sS -H "Authorization: bearer ${TOKEN}" -H "Fiware-Service: ${FIWARE_SERVICE}" -H "Fiware-Servicepath: ${DELIVERY_ROBOT_SERVICEPATH}" http://${HOST_IPADDR}:8080/idas/json/manage/iot/services/ | jq .

example)
```json
{
  "count": 1,
  "services": [
    {
      "commands": [],
      "lazy": [],
      "attributes": [],
      "_id": "5d1daefe35d6f5000f7e1ed5",
      "resource": "/iot/json",
      "apikey": "delivery_robot",
      "service": "uoa_poc2",
      "subservice": "/delivery_robot",
      "__v": 0,
      "static_attributes": [],
      "internal_attributes": [],
      "entity_type": "delivery_robot"
    }
  ]
}
```

## register delivery robot device (megarover_01)

In [ ]:
TOKEN=$(cat ${CORE_ROOT}/secrets/auth-tokens.json | jq '.[0].settings.bearer_tokens[0].token' -r)
curl -H "Authorization: bearer ${TOKEN}" -H "Fiware-Service: ${FIWARE_SERVICE}" -H "Fiware-ServicePath: ${DELIVERY_ROBOT_SERVICEPATH}" -H "Content-Type: application/json" http://${HOST_IPADDR}:8080/idas/json/manage/iot/devices/ -X POST -d @- <<__EOS__
{
  "devices": [
    {
      "device_id": "${DELIVERY_ROBOT_01}",
      "entity_name": "${DELIVERY_ROBOT_01}",
      "entity_type": "${DELIVERY_ROBOT_TYPE}",
      "timezone": "Asia/Tokyo",
      "protocol": "json",
      "attributes": [
        {
          "name": "time",
          "type": "ISO8601"
        },
        {
          "name": "mode",
          "type": "string"
        },
        {
          "name": "errors",
          "type": "array"
        },
        {
          "name": "pose",
          "type": "object"
        },
        {
          "name": "destination",
          "type": "object"
        },
        {
          "name": "covariance",
          "type": "array"
        },
        {
          "name": "battery",
          "type": "object"
        },
        {
          "name": "current_routes",
          "type": "array"
        },
        {
          "name": "navigating_waypoints",
          "type": "object"
        },
        {
          "name": "remaining_waypoints_list",
          "type": "array"
        }
      ],
      "commands": [
        {
          "name": "send_cmd",
          "type": "object"
        }
      ],
      "transport": "AMQP"
    }
  ]
}
__EOS__

expected)
```json
{}
```

### confirm registered device

In [ ]:
TOKEN=$(cat ${CORE_ROOT}/secrets/auth-tokens.json | jq '.[0].settings.bearer_tokens[0].token' -r)
curl -sS -H "Authorization: bearer ${TOKEN}" -H "Fiware-Service: ${FIWARE_SERVICE}" -H "Fiware-Servicepath: ${DELIVERY_ROBOT_SERVICEPATH}" http://${HOST_IPADDR}:8080/idas/json/manage/iot/devices/${DELIVERY_ROBOT_01}/ | jq .

example)
```json
{
  "device_id": "megarover_01",
  "service": "uoa_poc2",
  "service_path": "/delivery_robot",
  "entity_name": "megarover_01",
  "entity_type": "delivery_robot",
  "transport": "AMQP",
  "attributes": [
    {
      "object_id": "time",
      "name": "time",
      "type": "ISO8601"
    },
    {
      "object_id": "mode",
      "name": "mode",
      "type": "string"
    },
    {
      "object_id": "errors",
      "name": "errors",
      "type": "array"
    },
    {
      "object_id": "pose",
      "name": "pose",
      "type": "object"
    },
    {
      "object_id": "destination",
      "name": "destination",
      "type": "object"
    },
    {
      "object_id": "covariance",
      "name": "covariance",
      "type": "array"
    },
    {
      "object_id": "battery",
      "name": "battery",
      "type": "object"
    },
    {
      "object_id": "current_routes",
      "name": "current_routes",
      "type": "array"
    },
    {
      "object_id": "navigating_waypoints",
      "name": "navigating_waypoints",
      "type": "object"
    },
    {
      "object_id": "remaining_waypoints_list",
      "name": "remaining_waypoints_list",
      "type": "array"
    }
  ],
  "lazy": [],
  "commands": [
    {
      "object_id": "send_cmd",
      "name": "send_cmd",
      "type": "object"
    }
  ],
  "static_attributes": [],
  "protocol": "json"
}
```

In [ ]:
TOKEN=$(cat ${CORE_ROOT}/secrets/auth-tokens.json | jq '.[0].settings.bearer_tokens[0].token' -r)
curl -sS -H "Authorization: bearer ${TOKEN}" -H "Fiware-Service: ${FIWARE_SERVICE}" -H "Fiware-Servicepath: ${DELIVERY_ROBOT_SERVICEPATH}" http://${HOST_IPADDR}:8080/orion/v2/entities/${DELIVERY_ROBOT_01}/ | jq .

example)
```json
{
  "id": "megarover_01",
  "type": "delivery_robot",
  "TimeInstant": {
    "type": "ISO8601",
    "value": " ",
    "metadata": {}
  },
  "battery": {
    "type": "object",
    "value": " ",
    "metadata": {}
  },
  "covariance": {
    "type": "array",
    "value": " ",
    "metadata": {}
  },
  "current_routes": {
    "type": "array",
    "value": " ",
    "metadata": {}
  },
  "destination": {
    "type": "object",
    "value": " ",
    "metadata": {}
  },
  "errors": {
    "type": "array",
    "value": " ",
    "metadata": {}
  },
  "mode": {
    "type": "string",
    "value": " ",
    "metadata": {}
  },
  "navigating_waypoints": {
    "type": "object",
    "value": " ",
    "metadata": {}
  },
  "pose": {
    "type": "object",
    "value": " ",
    "metadata": {}
  },
  "remaining_waypoints_list": {
    "type": "array",
    "value": " ",
    "metadata": {}
  },
  "send_cmd_info": {
    "type": "commandResult",
    "value": " ",
    "metadata": {}
  },
  "send_cmd_status": {
    "type": "commandStatus",
    "value": "UNKNOWN",
    "metadata": {}
  },
  "time": {
    "type": "ISO8601",
    "value": " ",
    "metadata": {}
  },
  "send_cmd": {
    "type": "object",
    "value": "",
    "metadata": {}
  }
}
```

### register `cygnus-mongo` as a subscriber of delivery robot device

In [ ]:
TOKEN=$(cat ${CORE_ROOT}/secrets/auth-tokens.json | jq '.[0].settings.bearer_tokens[0].token' -r)
curl -i -H "Authorization: bearer ${TOKEN}" -H "Fiware-Service: ${FIWARE_SERVICE}" -H "Fiware-ServicePath: ${DELIVERY_ROBOT_SERVICEPATH}" -H "Content-Type: application/json" http://${HOST_IPADDR}:8080/orion/v2/subscriptions/ -X POST -d @- <<__EOS__
{
  "subject": {
    "entities": [{
      "idPattern": "${DELIVERY_ROBOT_ID_PREFIX}.*",
      "type": "${DELIVERY_ROBOT_TYPE}"
    }],
    "condition": {
      "attrs": ["time", "mode", "errors", "pose", "destination", "covariance", "battery"]
    }
  },
  "notification": {
    "http": {
      "url": "http://cygnus-mongo:5050/notify"
    },
    "attrs": ["time", "mode", "errors", "pose", "destination", "covariance", "battery"],
    "attrsFormat": "legacy"
  }
}
__EOS__

example)
```
HTTP/1.1 201 Created
content-length: 0
location: /v2/subscriptions/5d68c139eb5d293223fb3ec8
fiware-correlator: e2feba3c-caee-11e9-9ea6-0242ac110012
date: Fri, 30 Aug 2019 06:24:56 GMT
x-envoy-upstream-service-time: 15
server: envoy
```

In [ ]:
TOKEN=$(cat ${CORE_ROOT}/secrets/auth-tokens.json | jq '.[0].settings.bearer_tokens[0].token' -r)
curl -i -H "Authorization: bearer ${TOKEN}" -H "Fiware-Service: ${FIWARE_SERVICE}" -H "Fiware-ServicePath: ${DELIVERY_ROBOT_SERVICEPATH}" -H "Content-Type: application/json" http://${HOST_IPADDR}:8080/orion/v2/subscriptions/ -X POST -d @- <<__EOS__
{
  "subject": {
    "entities": [{
      "idPattern": "${DELIVERY_ROBOT_ID_PREFIX}.*",
      "type": "${DELIVERY_ROBOT_TYPE}"
    }],
    "condition": {
      "attrs": ["send_cmd", "send_cmd_status", "send_cmd_info"]
    }
  },
  "notification": {
    "http": {
      "url": "http://cygnus-mongo:5050/notify"
    },
    "attrs": ["send_cmd", "send_cmd_status", "send_cmd_info"],
    "attrsFormat": "legacy"
  }
}
__EOS__

example)
```
HTTP/1.1 201 Created
content-length: 0
location: /v2/subscriptions/5d68c14beb5d293223fb3ec9
fiware-correlator: edb0662e-caee-11e9-9ea6-0242ac110012
date: Fri, 30 Aug 2019 06:25:14 GMT
x-envoy-upstream-service-time: 3
server: envoy
```

In [ ]:
TOKEN=$(cat ${CORE_ROOT}/secrets/auth-tokens.json | jq '.[0].settings.bearer_tokens[0].token' -r)
curl -i -H "Authorization: bearer ${TOKEN}" -H "Fiware-Service: ${FIWARE_SERVICE}" -H "Fiware-ServicePath: ${DELIVERY_ROBOT_SERVICEPATH}" -H "Content-Type: application/json" http://${HOST_IPADDR}:8080/orion/v2/subscriptions/ -X POST -d @- <<__EOS__
{
  "subject": {
    "entities": [{
      "idPattern": "${DELIVERY_ROBOT_ID_PREFIX}.*",
      "type": "${DELIVERY_ROBOT_TYPE}"
    }],
    "condition": {
      "attrs": ["current_routes", "navigating_waypoints", "remaining_waypoints_list"]
    }
  },
  "notification": {
    "http": {
      "url": "http://cygnus-mongo:5050/notify"
    },
    "attrs": ["current_routes", "navigating_waypoints", "remaining_waypoints_list"],
    "attrsFormat": "legacy"
  }
}
__EOS__

example)

```
HTTP/1.1 201 Created
content-length: 0
location: /v2/subscriptions/5d68c15ba91e89ca4bbf41d2
fiware-correlator: f75bc5e2-caee-11e9-9423-0242ac110013
date: Fri, 30 Aug 2019 06:25:30 GMT
x-envoy-upstream-service-time: 10
server: envoy

```

In [ ]:
TOKEN=$(cat ${CORE_ROOT}/secrets/auth-tokens.json | jq '.[0].settings.bearer_tokens[0].token' -r)
curl -sS -H "Authorization: bearer ${TOKEN}" -H "Fiware-Service: ${FIWARE_SERVICE}" -H "Fiware-ServicePath: ${DELIVERY_ROBOT_SERVICEPATH}" http://${HOST_IPADDR}:8080/orion/v2/subscriptions/ | jq .

example)
```json
[
  {
    "id": "5d68c139eb5d293223fb3ec8",
    "status": "active",
    "subject": {
      "entities": [
        {
          "idPattern": "megarover_.*",
          "type": "delivery_robot"
        }
      ],
      "condition": {
        "attrs": [
          "time",
          "mode",
          "errors",
          "pose",
          "destination",
          "covariance",
          "battery"
        ]
      }
    },
    "notification": {
      "timesSent": 1,
      "lastNotification": "2019-08-30T06:24:57.00Z",
      "attrs": [
        "time",
        "mode",
        "errors",
        "pose",
        "destination",
        "covariance",
        "battery"
      ],
      "attrsFormat": "legacy",
      "http": {
        "url": "http://cygnus-mongo:5050/notify"
      },
      "lastSuccess": "2019-08-30T06:24:57.00Z",
      "lastSuccessCode": 200
    }
  },
  {
    "id": "5d68c14beb5d293223fb3ec9",
    "status": "active",
    "subject": {
      "entities": [
        {
          "idPattern": "megarover_.*",
          "type": "delivery_robot"
        }
      ],
      "condition": {
        "attrs": [
          "send_cmd",
          "send_cmd_status",
          "send_cmd_info"
        ]
      }
    },
    "notification": {
      "timesSent": 1,
      "lastNotification": "2019-08-30T06:25:15.00Z",
      "attrs": [
        "send_cmd",
        "send_cmd_status",
        "send_cmd_info"
      ],
      "attrsFormat": "legacy",
      "http": {
        "url": "http://cygnus-mongo:5050/notify"
      },
      "lastSuccess": "2019-08-30T06:25:15.00Z",
      "lastSuccessCode": 200
    }
  },
  {
    "id": "5d68c15ba91e89ca4bbf41d2",
    "status": "active",
    "subject": {
      "entities": [
        {
          "idPattern": "megarover_.*",
          "type": "delivery_robot"
        }
      ],
      "condition": {
        "attrs": [
          "current_routes",
          "navigating_waypoints",
          "remaining_waypoints_list"
        ]
      }
    },
    "notification": {
      "timesSent": 1,
      "lastNotification": "2019-08-30T06:25:31.00Z",
      "attrs": [
        "current_routes",
        "navigating_waypoints",
        "remaining_waypoints_list"
      ],
      "attrsFormat": "legacy",
      "http": {
        "url": "http://cygnus-mongo:5050/notify"
      },
      "lastSuccess": "2019-08-30T06:25:31.00Z",
      "lastSuccessCode": 200
    }
  }
]
```

## test subscribing the attributes of delivery robot (megarover_01)

### prepare a command to subscribe all topics

In [ ]:
echo "mosquitto_sub -h ${HOST_IPADDR} -p 1883 -d -u iotagent -P ${MQTT__iotagent} -t /#"

### subscribe all topics

_Outside of this notebook_
1. open a ternminal.
1. run the above command displayed `prepare a command to subscribe all topics`.

### publish a MQTT message to emulate delivery robot state

In [ ]:
m=$(cat << __EOS__ 
{
  "time": "$(now)",
  "mode": "navi",
  "errors": [],
  "pose": {
    "point": { "x": 3.402, "y": 1.015, "z": -0.002 },
    "angle": { "roll": -0.001, "pitch": 0.012, "yaw": 1.581 }
  },
  "destination": {
    "point": { "x": 3.411, "y": 2.81, "z": 0.0 },
    "angle_optional": {
      "valid": true,
      "angle": { "roll": 0.0, "pitch": 0.0, "yaw": 1.571 }
    }
  },
  "covariance": [
    0.1, 0.0, 0.0, 0.0, 0.0, 0.0,
    0.0, 0.1, 0.0, 0.0, 0.0, 0.0,
    0.0, 0.0, 1.7976931348623157e+308, 0.0, 0.0, 0.0,
    0.0, 0.0, 0.0, 1.7976931348623157e+308, 0.0, 0.0,
    0.0, 0.0, 0.0, 0.0, 1.7976931348623157e+308, 0.0,
    0.0, 0.0, 0.0, 0.0, 0.0, 0.05
  ],
  "battery": {
    "voltage": 11.495,
    "current_optional": {
      "valid": true,
      "current": 0.23
    }
  }
}
__EOS__
)
mosquitto_pub -h ${HOST_IPADDR} -p 1883 -d -u iotagent -P ${MQTT__iotagent} -t /${DELIVERY_ROBOT_TYPE}/${DELIVERY_ROBOT_01}/attrs -m "$m"

### confirm the topic

when executing the above command, show below messages on the opened terminal.

example)
```
Client mosq/UipAGUSYM6pHegL2WR received PUBLISH (d0, q0, r0, m0, '/delivery_robot/megarover_01/attrs', ... (804 bytes))
{
  "time": "2019-08-30T15:31:04.520421+09:00",
  "mode": "navi",
  "errors": [],
  "pose": {
    "point": { "x": 3.402, "y": 1.015, "z": -0.002 },
    "angle": { "roll": -0.001, "pitch": 0.012, "yaw": 1.581 }
  },
  "destination": {
    "point": { "x": 3.411, "y": 2.81, "z": 0.0 },
    "angle_optional": {
      "valid": true,
      "angle": { "roll": 0.0, "pitch": 0.0, "yaw": 1.571 }
    }
  },
  "covariance": [
    0.1, 0.0, 0.0, 0.0, 0.0, 0.0,
    0.0, 0.1, 0.0, 0.0, 0.0, 0.0,
    0.0, 0.0, 1.7976931348623157e+308, 0.0, 0.0, 0.0,
    0.0, 0.0, 0.0, 1.7976931348623157e+308, 0.0, 0.0,
    0.0, 0.0, 0.0, 0.0, 1.7976931348623157e+308, 0.0,
    0.0, 0.0, 0.0, 0.0, 0.0, 0.05
  ],
  "battery": {
    "voltage": 11.495,
    "current_optional": {
      "valid": true,
      "current": 0.23
    }
  }
}
```

### confirm delivery robot entity (megarover_01)

In [ ]:
TOKEN=$(cat ${CORE_ROOT}/secrets/auth-tokens.json | jq '.[0].settings.bearer_tokens[0].token' -r)
curl -sS -H "Authorization: bearer ${TOKEN}" -H "Fiware-Service: ${FIWARE_SERVICE}" -H "Fiware-Servicepath: ${DELIVERY_ROBOT_SERVICEPATH}" http://${HOST_IPADDR}:8080/orion/v2/entities/${DELIVERY_ROBOT_01}/ | jq .

example)
```json
{
  "id": "megarover_01",
  "type": "delivery_robot",
  "TimeInstant": {
    "type": "ISO8601",
    "value": "2019-08-30T06:31:04.00Z",
    "metadata": {}
  },
  "battery": {
    "type": "object",
    "value": {
      "voltage": "11.495",
      "current_optional": {
        "valid": "true",
        "current": "0.23"
      }
    },
    "metadata": {
      "TimeInstant": {
        "type": "ISO8601",
        "value": "2019-08-30T06:31:04.555Z"
      }
    }
  },
  "covariance": {
    "type": "array",
    "value": [
      "0.1",
      "0",
      "0",
      "0",
      "0",
      "0",
      "0",
      "0.1",
      "0",
      "0",
      "0",
      "0",
      "0",
      "0",
      "1.7976931348623157e+308",
      "0",
      "0",
      "0",
      "0",
      "0",
      "0",
      "1.7976931348623157e+308",
      "0",
      "0",
      "0",
      "0",
      "0",
      "0",
      "1.7976931348623157e+308",
      "0",
      "0",
      "0",
      "0",
      "0",
      "0",
      "0.05"
    ],
    "metadata": {
      "TimeInstant": {
        "type": "ISO8601",
        "value": "2019-08-30T06:31:04.555Z"
      }
    }
  },
  "current_routes": {
    "type": "array",
    "value": " ",
    "metadata": {}
  },
  "destination": {
    "type": "object",
    "value": {
      "point": {
        "x": "3.411",
        "y": "2.81",
        "z": "0"
      },
      "angle_optional": {
        "valid": "true",
        "angle": {
          "roll": "0",
          "pitch": "0",
          "yaw": "1.571"
        }
      }
    },
    "metadata": {
      "TimeInstant": {
        "type": "ISO8601",
        "value": "2019-08-30T06:31:04.555Z"
      }
    }
  },
  "errors": {
    "type": "array",
    "value": "",
    "metadata": {
      "TimeInstant": {
        "type": "ISO8601",
        "value": "2019-08-30T06:31:04.555Z"
      }
    }
  },
  "mode": {
    "type": "string",
    "value": "navi",
    "metadata": {
      "TimeInstant": {
        "type": "ISO8601",
        "value": "2019-08-30T06:31:04.555Z"
      }
    }
  },
  "navigating_waypoints": {
    "type": "object",
    "value": " ",
    "metadata": {}
  },
  "pose": {
    "type": "object",
    "value": {
      "point": {
        "x": "3.402",
        "y": "1.015",
        "z": "-0.002"
      },
      "angle": {
        "roll": "-0.001",
        "pitch": "0.012",
        "yaw": "1.581"
      }
    },
    "metadata": {
      "TimeInstant": {
        "type": "ISO8601",
        "value": "2019-08-30T06:31:04.555Z"
      }
    }
  },
  "remaining_waypoints_list": {
    "type": "array",
    "value": " ",
    "metadata": {}
  },
  "send_cmd_info": {
    "type": "commandResult",
    "value": " ",
    "metadata": {}
  },
  "send_cmd_status": {
    "type": "commandStatus",
    "value": "UNKNOWN",
    "metadata": {}
  },
  "time": {
    "type": "ISO8601",
    "value": "2019-08-30T15:31:04.520421+09:00",
    "metadata": {
      "TimeInstant": {
        "type": "ISO8601",
        "value": "2019-08-30T06:31:04.555Z"
      }
    }
  },
  "send_cmd": {
    "type": "object",
    "value": "",
    "metadata": {}
  }
}
```

### confirm cygnus-mongo

In [ ]:
c=$(echo sth_${DELIVERY_ROBOT_SERVICEPATH}xffff${DELIVERY_ROBOT_01}xffff${DELIVERY_ROBOT_TYPE} | perl -pe 's/\//x002f/g; s/\$/x0024/g; s/=/xffff/g;')
kubectl exec mongodb-0 -c mongodb-replicaset -- mongo sth_${FIWARE_SERVICE} --eval "db.getCollection(\"${c}\").find().sort({recvTime: -1})"

example)
```
MongoDB shell version v4.1.13
connecting to: mongodb://127.0.0.1:27017/sth_uoa_poc2?compressors=disabled&gssapiServiceName=mongodb
Implicit session: session { "id" : UUID("144deaf9-6c17-42c9-9533-18b8aa00b3b7") }
MongoDB server version: 4.1.13
{ "_id" : ObjectId("5d68c2a80c76920014e606b8"), "recvTime" : ISODate("2019-08-30T06:31:04.555Z"), "attrName" : "time", "attrType" : "ISO8601", "attrValue" : "2019-08-30T15:31:04.520421+09:00" }
{ "_id" : ObjectId("5d68c2a80c76920014e606b9"), "recvTime" : ISODate("2019-08-30T06:31:04.555Z"), "attrName" : "mode", "attrType" : "string", "attrValue" : "navi" }
{ "_id" : ObjectId("5d68c2a80c76920014e606ba"), "recvTime" : ISODate("2019-08-30T06:31:04.555Z"), "attrName" : "pose", "attrType" : "object", "attrValue" : "{\"point\":{\"x\":\"3.402\",\"y\":\"1.015\",\"z\":\"-0.002\"},\"angle\":{\"roll\":\"-0.001\",\"pitch\":\"0.012\",\"yaw\":\"1.581\"}}" }
{ "_id" : ObjectId("5d68c2a80c76920014e606bb"), "recvTime" : ISODate("2019-08-30T06:31:04.555Z"), "attrName" : "destination", "attrType" : "object", "attrValue" : "{\"point\":{\"x\":\"3.411\",\"y\":\"2.81\",\"z\":\"0\"},\"angle_optional\":{\"valid\":\"true\",\"angle\":{\"roll\":\"0\",\"pitch\":\"0\",\"yaw\":\"1.571\"}}}" }
{ "_id" : ObjectId("5d68c2a80c76920014e606bc"), "recvTime" : ISODate("2019-08-30T06:31:04.555Z"), "attrName" : "covariance", "attrType" : "array", "attrValue" : "[\"0.1\",\"0\",\"0\",\"0\",\"0\",\"0\",\"0\",\"0.1\",\"0\",\"0\",\"0\",\"0\",\"0\",\"0\",\"1.7976931348623157e+308\",\"0\",\"0\",\"0\",\"0\",\"0\",\"0\",\"1.7976931348623157e+308\",\"0\",\"0\",\"0\",\"0\",\"0\",\"0\",\"1.7976931348623157e+308\",\"0\",\"0\",\"0\",\"0\",\"0\",\"0\",\"0.05\"]" }
{ "_id" : ObjectId("5d68c2a80c76920014e606bd"), "recvTime" : ISODate("2019-08-30T06:31:04.555Z"), "attrName" : "battery", "attrType" : "object", "attrValue" : "{\"voltage\":\"11.495\",\"current_optional\":{\"valid\":\"true\",\"current\":\"0.23\"}}" }
{ "_id" : ObjectId("5d68c14e06699000149c12cd"), "recvTime" : ISODate("2019-08-30T06:25:15.046Z"), "attrName" : "send_cmd_status", "attrType" : "commandStatus", "attrValue" : "UNKNOWN" }
```

### confirm sth-comet

In [ ]:
TOKEN=$(cat ${CORE_ROOT}/secrets/auth-tokens.json | jq '.[0].settings.bearer_tokens[0].token' -r)
for a in "time" "mode" "pose" "destination" "covariance" "battery"; do
  curl -sS -H "Authorization: bearer ${TOKEN}" -H "Fiware-Service: ${FIWARE_SERVICE}" -H "Fiware-ServicePath: ${DELIVERY_ROBOT_SERVICEPATH}" http://${HOST_IPADDR}:8080/comet/STH/v1/contextEntities/type/${DELIVERY_ROBOT_TYPE}/id/${DELIVERY_ROBOT_01}/attributes/${a}?lastN=100 | jq .
done

example)
```json
{
  "contextResponses": [
    {
      "contextElement": {
        "attributes": [
          {
            "name": "time",
            "values": [
              {
                "recvTime": "2019-08-30T06:31:04.555Z",
                "attrType": "ISO8601",
                "attrValue": "2019-08-30T15:31:04.520421+09:00"
              }
            ]
          }
        ],
        "id": "megarover_01",
        "isPattern": false,
        "type": "delivery_robot"
      },
      "statusCode": {
        "code": "200",
        "reasonPhrase": "OK"
      }
    }
  ]
}
```
```json
{
  "contextResponses": [
    {
      "contextElement": {
        "attributes": [
          {
            "name": "mode",
            "values": [
              {
                "recvTime": "2019-08-30T06:31:04.555Z",
                "attrType": "string",
                "attrValue": "navi"
              }
            ]
          }
        ],
        "id": "megarover_01",
        "isPattern": false,
        "type": "delivery_robot"
      },
      "statusCode": {
        "code": "200",
        "reasonPhrase": "OK"
      }
    }
  ]
}
```
```json
{
  "contextResponses": [
    {
      "contextElement": {
        "attributes": [
          {
            "name": "pose",
            "values": [
              {
                "recvTime": "2019-08-30T06:31:04.555Z",
                "attrType": "object",
                "attrValue": "{\"point\":{\"x\":\"3.402\",\"y\":\"1.015\",\"z\":\"-0.002\"},\"angle\":{\"roll\":\"-0.001\",\"pitch\":\"0.012\",\"yaw\":\"1.581\"}}"
              }
            ]
          }
        ],
        "id": "megarover_01",
        "isPattern": false,
        "type": "delivery_robot"
      },
      "statusCode": {
        "code": "200",
        "reasonPhrase": "OK"
      }
    }
  ]
}
```
```json
{
  "contextResponses": [
    {
      "contextElement": {
        "attributes": [
          {
            "name": "destination",
            "values": [
              {
                "recvTime": "2019-08-30T06:31:04.555Z",
                "attrType": "object",
                "attrValue": "{\"point\":{\"x\":\"3.411\",\"y\":\"2.81\",\"z\":\"0\"},\"angle_optional\":{\"valid\":\"true\",\"angle\":{\"roll\":\"0\",\"pitch\":\"0\",\"yaw\":\"1.571\"}}}"
              }
            ]
          }
        ],
        "id": "megarover_01",
        "isPattern": false,
        "type": "delivery_robot"
      },
      "statusCode": {
        "code": "200",
        "reasonPhrase": "OK"
      }
    }
  ]
}
```
```json
{
  "contextResponses": [
    {
      "contextElement": {
        "attributes": [
          {
            "name": "covariance",
            "values": [
              {
                "recvTime": "2019-08-30T06:31:04.555Z",
                "attrType": "array",
                "attrValue": "[\"0.1\",\"0\",\"0\",\"0\",\"0\",\"0\",\"0\",\"0.1\",\"0\",\"0\",\"0\",\"0\",\"0\",\"0\",\"1.7976931348623157e+308\",\"0\",\"0\",\"0\",\"0\",\"0\",\"0\",\"1.7976931348623157e+308\",\"0\",\"0\",\"0\",\"0\",\"0\",\"0\",\"1.7976931348623157e+308\",\"0\",\"0\",\"0\",\"0\",\"0\",\"0\",\"0.05\"]"
              }
            ]
          }
        ],
        "id": "megarover_01",
        "isPattern": false,
        "type": "delivery_robot"
      },
      "statusCode": {
        "code": "200",
        "reasonPhrase": "OK"
      }
    }
  ]
}
```
```json
{
  "contextResponses": [
    {
      "contextElement": {
        "attributes": [
          {
            "name": "battery",
            "values": [
              {
                "recvTime": "2019-08-30T06:31:04.555Z",
                "attrType": "object",
                "attrValue": "{\"voltage\":\"11.495\",\"current_optional\":{\"valid\":\"true\",\"current\":\"0.23\"}}"
              }
            ]
          }
        ],
        "id": "megarover_01",
        "isPattern": false,
        "type": "delivery_robot"
      },
      "statusCode": {
        "code": "200",
        "reasonPhrase": "OK"
      }
    }
  ]
}
```

## test publishing the `send_cmd` command of delivery robot (megarover_01)

### prepare a command to subscribe all topics

In [ ]:
echo "mosquitto_sub -h ${HOST_IPADDR} -p 1883 -d -u iotagent -P ${MQTT__iotagent} -t /#"

### subscribe all topics

_Outside of this notebook_
1. open a ternminal.
1. run the above command displayed `prepare a command to subscribe all topics`.

### send a NGSI message to emulate sending command

In [ ]:
TOKEN=$(cat ${CORE_ROOT}/secrets/auth-tokens.json | jq '.[0].settings.bearer_tokens[0].token' -r)
curl -i -H "Authorization: bearer ${TOKEN}" -H "Fiware-Service: ${FIWARE_SERVICE}" -H "Fiware-Servicepath: ${DELIVERY_ROBOT_SERVICEPATH}" -H "Content-Type: application/json" http://${HOST_IPADDR}:8080/orion/v2/entities/${DELIVERY_ROBOT_01}/attrs?type=${DELIVERY_ROBOT_TYPE} -X PATCH -d @-<<__EOS__
{
  "send_cmd": {
    "value": {
      "time": "$(now)",
      "cmd": "navi",
      "waypoints": [
        {
          "point": { "x": 0.503, "y": 0.0, "z": 0.0 },
          "angle_optional": {
            "valid": false,
            "angle": { "roll": 0.0, "pitch": 0.0, "yaw": 0.0 }
          }
        },
        {
          "point": { "x": 3.411, "y": 0.0, "z": 0.0 },
          "angle_optional": {
            "valid": false,
            "angle": { "roll": 0.0, "pitch": 0.0, "yaw": 0.0 }
          }
        },
        {
          "point": { "x": 3.411, "y": 2.81, "z": 0.0 },
          "angle_optional": {
            "valid": true,
            "angle": { "roll": 0.0, "pitch": 0.0, "yaw": 1.571 }
          }
        }
      ]
    }
  }
}
__EOS__

example)
```
HTTP/1.1 204 No Content
content-length: 0
fiware-correlator: 4681346c-caf0-11e9-8544-0242ac110012
date: Fri, 30 Aug 2019 06:34:53 GMT
x-envoy-upstream-service-time: 57
server: envoy
```

### confirm the topic

when executing the above command, show below messages on the opened terminal.

example)
```
Client mosq/UipAGUSYM6pHegL2WR received PUBLISH (d0, q0, r0, m0, '/delivery_robot/megarover_01/cmd', ... (401 bytes))
{"send_cmd":{"time":"2019-08-30T15:34:53.605158+09:00","cmd":"navi","waypoints":[{"point":{"x":0.503,"y":0,"z":0},"angle_optional":{"valid":false,"angle":{"roll":0,"pitch":0,"yaw":0}}},{"point":{"x":3.411,"y":0,"z":0},"angle_optional":{"valid":false,"angle":{"roll":0,"pitch":0,"yaw":0}}},{"point":{"x":3.411,"y":2.81,"z":0},"angle_optional":{"valid":true,"angle":{"roll":0,"pitch":0,"yaw":1.571}}}]}}
```

### confirm delivery robot entity (megarover_01)

In [ ]:
TOKEN=$(cat ${CORE_ROOT}/secrets/auth-tokens.json | jq '.[0].settings.bearer_tokens[0].token' -r)
curl -sS -H "Authorization: bearer ${TOKEN}" -H "Fiware-Service: ${FIWARE_SERVICE}" -H "Fiware-Servicepath: ${DELIVERY_ROBOT_SERVICEPATH}" http://${HOST_IPADDR}:8080/orion/v2/entities/${DELIVERY_ROBOT_01}/ | jq .

`send_cmd_status` is `PENDING`

example)
```json
{
  "id": "megarover_01",
  "type": "delivery_robot",
  "TimeInstant": {
    "type": "ISO8601",
    "value": "2019-08-30T06:34:53.00Z",
    "metadata": {}
  },
  "battery": {
    "type": "object",
    "value": {
      "voltage": "11.495",
      "current_optional": {
        "valid": "true",
        "current": "0.23"
      }
    },
    "metadata": {
      "TimeInstant": {
        "type": "ISO8601",
        "value": "2019-08-30T06:31:04.555Z"
      }
    }
  },
  "covariance": {
    "type": "array",
    "value": [
      "0.1",
      "0",
      "0",
      "0",
      "0",
      "0",
      "0",
      "0.1",
      "0",
      "0",
      "0",
      "0",
      "0",
      "0",
      "1.7976931348623157e+308",
      "0",
      "0",
      "0",
      "0",
      "0",
      "0",
      "1.7976931348623157e+308",
      "0",
      "0",
      "0",
      "0",
      "0",
      "0",
      "1.7976931348623157e+308",
      "0",
      "0",
      "0",
      "0",
      "0",
      "0",
      "0.05"
    ],
    "metadata": {
      "TimeInstant": {
        "type": "ISO8601",
        "value": "2019-08-30T06:31:04.555Z"
      }
    }
  },
  "current_routes": {
    "type": "array",
    "value": " ",
    "metadata": {}
  },
  "destination": {
    "type": "object",
    "value": {
      "point": {
        "x": "3.411",
        "y": "2.81",
        "z": "0"
      },
      "angle_optional": {
        "valid": "true",
        "angle": {
          "roll": "0",
          "pitch": "0",
          "yaw": "1.571"
        }
      }
    },
    "metadata": {
      "TimeInstant": {
        "type": "ISO8601",
        "value": "2019-08-30T06:31:04.555Z"
      }
    }
  },
  "errors": {
    "type": "array",
    "value": "",
    "metadata": {
      "TimeInstant": {
        "type": "ISO8601",
        "value": "2019-08-30T06:31:04.555Z"
      }
    }
  },
  "mode": {
    "type": "string",
    "value": "navi",
    "metadata": {
      "TimeInstant": {
        "type": "ISO8601",
        "value": "2019-08-30T06:31:04.555Z"
      }
    }
  },
  "navigating_waypoints": {
    "type": "object",
    "value": " ",
    "metadata": {}
  },
  "pose": {
    "type": "object",
    "value": {
      "point": {
        "x": "3.402",
        "y": "1.015",
        "z": "-0.002"
      },
      "angle": {
        "roll": "-0.001",
        "pitch": "0.012",
        "yaw": "1.581"
      }
    },
    "metadata": {
      "TimeInstant": {
        "type": "ISO8601",
        "value": "2019-08-30T06:31:04.555Z"
      }
    }
  },
  "remaining_waypoints_list": {
    "type": "array",
    "value": " ",
    "metadata": {}
  },
  "send_cmd_info": {
    "type": "commandResult",
    "value": " ",
    "metadata": {}
  },
  "send_cmd_status": {
    "type": "commandStatus",
    "value": "PENDING",
    "metadata": {
      "TimeInstant": {
        "type": "ISO8601",
        "value": "2019-08-30T06:34:53.560Z"
      }
    }
  },
  "time": {
    "type": "ISO8601",
    "value": "2019-08-30T15:31:04.520421+09:00",
    "metadata": {
      "TimeInstant": {
        "type": "ISO8601",
        "value": "2019-08-30T06:31:04.555Z"
      }
    }
  },
  "send_cmd": {
    "type": "object",
    "value": "",
    "metadata": {}
  }
}
```

### publish a MQTT message to emulate receiving command result

In [ ]:
m=$(cat << __EOS__ 
{
  "send_cmd": {
    "time": "$(now)",
    "received_time": "2019-07-05T07:28:26.1562279306+09:00",
    "received_cmd": "navi",
    "received_waypoints": [
      {
        "point": { "x": 0.503, "y": 0.0, "z": 0.0 },
        "angle_optional": { "valid": false, "angle": { "roll": 0.0, "pitch": 0.0, "yaw": 0.0 }}
      },
      {
        "point": { "x": 3.411, "y": 0.0, "z": 0.0 },
        "angle_optional": { "valid": false, "angle": { "roll": 0.0, "pitch": 0.0, "yaw": 0.0 }}
      },
      {
        "point:": { "x": 3.411, "y": 2.81, "z": 0.0 },
        "angle_optional": { "valid": true, "angle": { "roll": 0.0, "pitch": 0.0, "yaw": 1.571 }}
      } 
    ],
    "result": "ack",
    "errors": []
  }
}
__EOS__
)
mosquitto_pub -h ${HOST_IPADDR} -p 1883 -d -u iotagent -P ${MQTT__iotagent} -t /${DELIVERY_ROBOT_TYPE}/${DELIVERY_ROBOT_01}/cmdexe -m "${m}"

### confirm the topic

when executing the above command, show below messages on the opened terminal.

example)
```
Client mosq/UipAGUSYM6pHegL2WR received PUBLISH (d0, q0, r0, m0, '/delivery_robot/megarover_01/cmdexe', ... (734 bytes))
{
  "send_cmd": {
    "time": "2019-08-30T15:45:26.839559+09:00",
    "received_time": "2019-07-05T07:28:26.1562279306+09:00",
    "received_cmd": "navi",
    "received_waypoints": [
      {
        "point": { "x": 0.503, "y": 0.0, "z": 0.0 },
        "angle_optional": { "valid": false, "angle": { "roll": 0.0, "pitch": 0.0, "yaw": 0.0 }}
      },
      {
        "point": { "x": 3.411, "y": 0.0, "z": 0.0 },
        "angle_optional": { "valid": false, "angle": { "roll": 0.0, "pitch": 0.0, "yaw": 0.0 }}
      },
      {
        "point:": { "x": 3.411, "y": 2.81, "z": 0.0 },
        "angle_optional": { "valid": true, "angle": { "roll": 0.0, "pitch": 0.0, "yaw": 1.571 }}
      }
    ],
    "result": "ack",
    "errors": []
  }
}
```

### confirm delivery robot entity (megarover_01)

In [ ]:
TOKEN=$(cat ${CORE_ROOT}/secrets/auth-tokens.json | jq '.[0].settings.bearer_tokens[0].token' -r)
curl -sS -H "Authorization: bearer ${TOKEN}" -H "Fiware-Service: ${FIWARE_SERVICE}" -H "Fiware-Servicepath: ${DELIVERY_ROBOT_SERVICEPATH}" http://${HOST_IPADDR}:8080/orion/v2/entities/${DELIVERY_ROBOT_01}/ | jq .

`send_cmd_status` turns to `OK` and `send_cmd_info` has the result object

example)
```json
{
  "id": "megarover_01",
  "type": "delivery_robot",
  "TimeInstant": {
    "type": "ISO8601",
    "value": "2019-08-30T06:45:26.00Z",
    "metadata": {}
  },
  "battery": {
    "type": "object",
    "value": {
      "voltage": "11.495",
      "current_optional": {
        "valid": "true",
        "current": "0.23"
      }
    },
    "metadata": {
      "TimeInstant": {
        "type": "ISO8601",
        "value": "2019-08-30T06:31:04.555Z"
      }
    }
  },
  "covariance": {
    "type": "array",
    "value": [
      "0.1",
      "0",
      "0",
      "0",
      "0",
      "0",
      "0",
      "0.1",
      "0",
      "0",
      "0",
      "0",
      "0",
      "0",
      "1.7976931348623157e+308",
      "0",
      "0",
      "0",
      "0",
      "0",
      "0",
      "1.7976931348623157e+308",
      "0",
      "0",
      "0",
      "0",
      "0",
      "0",
      "1.7976931348623157e+308",
      "0",
      "0",
      "0",
      "0",
      "0",
      "0",
      "0.05"
    ],
    "metadata": {
      "TimeInstant": {
        "type": "ISO8601",
        "value": "2019-08-30T06:31:04.555Z"
      }
    }
  },
  "current_routes": {
    "type": "array",
    "value": " ",
    "metadata": {}
  },
  "destination": {
    "type": "object",
    "value": {
      "point": {
        "x": "3.411",
        "y": "2.81",
        "z": "0"
      },
      "angle_optional": {
        "valid": "true",
        "angle": {
          "roll": "0",
          "pitch": "0",
          "yaw": "1.571"
        }
      }
    },
    "metadata": {
      "TimeInstant": {
        "type": "ISO8601",
        "value": "2019-08-30T06:31:04.555Z"
      }
    }
  },
  "errors": {
    "type": "array",
    "value": "",
    "metadata": {
      "TimeInstant": {
        "type": "ISO8601",
        "value": "2019-08-30T06:31:04.555Z"
      }
    }
  },
  "mode": {
    "type": "string",
    "value": "navi",
    "metadata": {
      "TimeInstant": {
        "type": "ISO8601",
        "value": "2019-08-30T06:31:04.555Z"
      }
    }
  },
  "navigating_waypoints": {
    "type": "object",
    "value": " ",
    "metadata": {}
  },
  "pose": {
    "type": "object",
    "value": {
      "point": {
        "x": "3.402",
        "y": "1.015",
        "z": "-0.002"
      },
      "angle": {
        "roll": "-0.001",
        "pitch": "0.012",
        "yaw": "1.581"
      }
    },
    "metadata": {
      "TimeInstant": {
        "type": "ISO8601",
        "value": "2019-08-30T06:31:04.555Z"
      }
    }
  },
  "remaining_waypoints_list": {
    "type": "array",
    "value": " ",
    "metadata": {}
  },
  "send_cmd_info": {
    "type": "commandResult",
    "value": {
      "time": "2019-08-30T15:45:26.839559+09:00",
      "received_time": "2019-07-05T07:28:26.1562279306+09:00",
      "received_cmd": "navi",
      "received_waypoints": [
        {
          "point": {
            "x": "0.503",
            "y": "0",
            "z": "0"
          },
          "angle_optional": {
            "valid": "false",
            "angle": {
              "roll": "0",
              "pitch": "0",
              "yaw": "0"
            }
          }
        },
        {
          "point": {
            "x": "3.411",
            "y": "0",
            "z": "0"
          },
          "angle_optional": {
            "valid": "false",
            "angle": {
              "roll": "0",
              "pitch": "0",
              "yaw": "0"
            }
          }
        },
        {
          "point:": {
            "x": "3.411",
            "y": "2.81",
            "z": "0"
          },
          "angle_optional": {
            "valid": "true",
            "angle": {
              "roll": "0",
              "pitch": "0",
              "yaw": "1.571"
            }
          }
        }
      ],
      "result": "ack",
      "errors": ""
    },
    "metadata": {
      "TimeInstant": {
        "type": "ISO8601",
        "value": "2019-08-30T06:45:26.870Z"
      }
    }
  },
  "send_cmd_status": {
    "type": "commandStatus",
    "value": "OK",
    "metadata": {
      "TimeInstant": {
        "type": "ISO8601",
        "value": "2019-08-30T06:45:26.870Z"
      }
    }
  },
  "time": {
    "type": "ISO8601",
    "value": "2019-08-30T15:31:04.520421+09:00",
    "metadata": {
      "TimeInstant": {
        "type": "ISO8601",
        "value": "2019-08-30T06:31:04.555Z"
      }
    }
  },
  "send_cmd": {
    "type": "object",
    "value": "",
    "metadata": {}
  }
}
```

### confirm cygnus-mongo

In [ ]:
c=$(echo sth_${DELIVERY_ROBOT_SERVICEPATH}xffff${DELIVERY_ROBOT_01}xffff${DELIVERY_ROBOT_TYPE} | perl -pe 's/\//x002f/g; s/\$/x0024/g; s/=/xffff/g;')
kubectl exec mongodb-0 -c mongodb-replicaset -- mongo sth_${FIWARE_SERVICE} --eval "db.getCollection(\"${c}\").find().sort({recvTime: -1})"

example)
```
MongoDB shell version v4.1.13
connecting to: mongodb://127.0.0.1:27017/sth_uoa_poc2?compressors=disabled&gssapiServiceName=mongodb
Implicit session: session { "id" : UUID("ee075293-f818-49a3-a23a-fb51f803473e") }
MongoDB server version: 4.1.13
{ "_id" : ObjectId("5d68c60706699000149c12cf"), "recvTime" : ISODate("2019-08-30T06:45:26.870Z"), "attrName" : "send_cmd_status", "attrType" : "commandStatus", "attrValue" : "OK" }
{ "_id" : ObjectId("5d68c60706699000149c12d0"), "recvTime" : ISODate("2019-08-30T06:45:26.870Z"), "attrName" : "send_cmd_info", "attrType" : "commandResult", "attrValue" : "{\"time\":\"2019-08-30T15:45:26.839559+09:00\",\"received_time\":\"2019-07-05T07:28:26.1562279306+09:00\",\"received_cmd\":\"navi\",\"received_waypoints\":[{\"point\":{\"x\":\"0.503\",\"y\":\"0\",\"z\":\"0\"},\"angle_optional\":{\"valid\":\"false\",\"angle\":{\"roll\":\"0\",\"pitch\":\"0\",\"yaw\":\"0\"}}},{\"point\":{\"x\":\"3.411\",\"y\":\"0\",\"z\":\"0\"},\"angle_optional\":{\"valid\":\"false\",\"angle\":{\"roll\":\"0\",\"pitch\":\"0\",\"yaw\":\"0\"}}},{\"point:\":{\"x\":\"3.411\",\"y\":\"2.81\",\"z\":\"0\"},\"angle_optional\":{\"valid\":\"true\",\"angle\":{\"roll\":\"0\",\"pitch\":\"0\",\"yaw\":\"1.571\"}}}],\"result\":\"ack\",\"errors\":\"\"}" }
{ "_id" : ObjectId("5d68c38d06699000149c12ce"), "recvTime" : ISODate("2019-08-30T06:34:53.560Z"), "attrName" : "send_cmd_status", "attrType" : "commandStatus", "attrValue" : "PENDING" }
{ "_id" : ObjectId("5d68c2a80c76920014e606b8"), "recvTime" : ISODate("2019-08-30T06:31:04.555Z"), "attrName" : "time", "attrType" : "ISO8601", "attrValue" : "2019-08-30T15:31:04.520421+09:00" }
{ "_id" : ObjectId("5d68c2a80c76920014e606b9"), "recvTime" : ISODate("2019-08-30T06:31:04.555Z"), "attrName" : "mode", "attrType" : "string", "attrValue" : "navi" }
{ "_id" : ObjectId("5d68c2a80c76920014e606ba"), "recvTime" : ISODate("2019-08-30T06:31:04.555Z"), "attrName" : "pose", "attrType" : "object", "attrValue" : "{\"point\":{\"x\":\"3.402\",\"y\":\"1.015\",\"z\":\"-0.002\"},\"angle\":{\"roll\":\"-0.001\",\"pitch\":\"0.012\",\"yaw\":\"1.581\"}}" }
{ "_id" : ObjectId("5d68c2a80c76920014e606bb"), "recvTime" : ISODate("2019-08-30T06:31:04.555Z"), "attrName" : "destination", "attrType" : "object", "attrValue" : "{\"point\":{\"x\":\"3.411\",\"y\":\"2.81\",\"z\":\"0\"},\"angle_optional\":{\"valid\":\"true\",\"angle\":{\"roll\":\"0\",\"pitch\":\"0\",\"yaw\":\"1.571\"}}}" }
{ "_id" : ObjectId("5d68c2a80c76920014e606bc"), "recvTime" : ISODate("2019-08-30T06:31:04.555Z"), "attrName" : "covariance", "attrType" : "array", "attrValue" : "[\"0.1\",\"0\",\"0\",\"0\",\"0\",\"0\",\"0\",\"0.1\",\"0\",\"0\",\"0\",\"0\",\"0\",\"0\",\"1.7976931348623157e+308\",\"0\",\"0\",\"0\",\"0\",\"0\",\"0\",\"1.7976931348623157e+308\",\"0\",\"0\",\"0\",\"0\",\"0\",\"0\",\"1.7976931348623157e+308\",\"0\",\"0\",\"0\",\"0\",\"0\",\"0\",\"0.05\"]" }
{ "_id" : ObjectId("5d68c2a80c76920014e606bd"), "recvTime" : ISODate("2019-08-30T06:31:04.555Z"), "attrName" : "battery", "attrType" : "object", "attrValue" : "{\"voltage\":\"11.495\",\"current_optional\":{\"valid\":\"true\",\"current\":\"0.23\"}}" }
{ "_id" : ObjectId("5d68c14e06699000149c12cd"), "recvTime" : ISODate("2019-08-30T06:25:15.046Z"), "attrName" : "send_cmd_status", "attrType" : "commandStatus", "attrValue" : "UNKNOWN" }
```

### confirm sth-comet

In [ ]:
TOKEN=$(cat ${CORE_ROOT}/secrets/auth-tokens.json | jq '.[0].settings.bearer_tokens[0].token' -r)
for a in "send_cmd" "send_cmd_status" "send_cmd_info"; do
  curl -sS -H "Authorization: bearer ${TOKEN}" -H "Fiware-Service: ${FIWARE_SERVICE}" -H "Fiware-ServicePath: ${DELIVERY_ROBOT_SERVICEPATH}" http://${HOST_IPADDR}:8080/comet/STH/v1/contextEntities/type/${DELIVERY_ROBOT_TYPE}/id/${DELIVERY_ROBOT_01}/attributes/${a}?lastN=100 | jq .
done

example)
```json
{
  "contextResponses": [
    {
      "contextElement": {
        "attributes": [
          {
            "name": "send_cmd",
            "values": []
          }
        ],
        "id": "megarover_01",
        "isPattern": false,
        "type": "delivery_robot"
      },
      "statusCode": {
        "code": "200",
        "reasonPhrase": "OK"
      }
    }
  ]
}
```
```json
{
  "contextResponses": [
    {
      "contextElement": {
        "attributes": [
          {
            "name": "send_cmd_status",
            "values": [
              {
                "recvTime": "2019-08-30T06:25:15.046Z",
                "attrType": "commandStatus",
                "attrValue": "UNKNOWN"
              },
              {
                "recvTime": "2019-08-30T06:34:53.560Z",
                "attrType": "commandStatus",
                "attrValue": "PENDING"
              },
              {
                "recvTime": "2019-08-30T06:45:26.870Z",
                "attrType": "commandStatus",
                "attrValue": "OK"
              }
            ]
          }
        ],
        "id": "megarover_01",
        "isPattern": false,
        "type": "delivery_robot"
      },
      "statusCode": {
        "code": "200",
        "reasonPhrase": "OK"
      }
    }
  ]
}
```
```json
{
  "contextResponses": [
    {
      "contextElement": {
        "attributes": [
          {
            "name": "send_cmd_info",
            "values": [
              {
                "recvTime": "2019-08-30T06:45:26.870Z",
                "attrType": "commandResult",
                "attrValue": "{\"time\":\"2019-08-30T15:45:26.839559+09:00\",\"received_time\":\"2019-07-05T07:28:26.1562279306+09:00\",\"received_cmd\":\"navi\",\"received_waypoints\":[{\"point\":{\"x\":\"0.503\",\"y\":\"0\",\"z\":\"0\"},\"angle_optional\":{\"valid\":\"false\",\"angle\":{\"roll\":\"0\",\"pitch\":\"0\",\"yaw\":\"0\"}}},{\"point\":{\"x\":\"3.411\",\"y\":\"0\",\"z\":\"0\"},\"angle_optional\":{\"valid\":\"false\",\"angle\":{\"roll\":\"0\",\"pitch\":\"0\",\"yaw\":\"0\"}}},{\"point:\":{\"x\":\"3.411\",\"y\":\"2.81\",\"z\":\"0\"},\"angle_optional\":{\"valid\":\"true\",\"angle\":{\"roll\":\"0\",\"pitch\":\"0\",\"yaw\":\"1.571\"}}}],\"result\":\"ack\",\"errors\":\"\"}"
              }
            ]
          }
        ],
        "id": "megarover_01",
        "isPattern": false,
        "type": "delivery_robot"
      },
      "statusCode": {
        "code": "200",
        "reasonPhrase": "OK"
      }
    }
  ]
}
```